# Preparing data for publication

In [1]:
# import libraries
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle as pkl
from IPython.display import display
import re
sns.set_theme(
    style="darkgrid",
    color_codes=True,
    palette='Dark2',
)

### DATA IMPORT & "TIDYING"

In [2]:
# import data from pickle files
pkl_files = ['species.pkl', 'species_long.pkl', 'species_summed.pkl']

# define import function
def import_pickled_data():
    dummy = []
    for file in pkl_files:
        with open('../../data/'+file, 'rb') as f:
            dummy.append(pkl.load(f))
    return dummy

species, species_long, species_summed = import_pickled_data()

In [3]:
# originally, species of the dataset were stored as excel sheets
sheets = ['Vanessa atalanta', 'Vanessa cardui', 'Inachis io', 'Issoria lathonia', 'Aglais urticae',
          'Aporia crataegi', 'Apatura ilia', 'Aphantopus hyperanthus', 'Araschnia levana', 'Nymphalis antiopa',
          'Nymphalis polychloros', 'Nymphalis xanthomelas', 'Papilio machaon', 'Polygonia c-album', 'Pararge aegeria']

# trap openings are facing either north- or south-wards
def get_trap_direction(trap):
    if re.fullmatch('L-(2|3|5|8)', trap):
        return 'north'
    elif re.fullmatch('L-(1|4|7)', trap):
        return 'south'
    else:
        return np.nan

# get data into tidy form!
# i.e., single dataframe with variables as columns and single observations as rows
all_species = pd.DataFrame()
for s in sheets:
    df = (
        species[s]
        .reset_index()
        # convert data from wide to long format
        .melt(id_vars=['datetime', 'Cloud', 'Temp', 'Wind'],
              value_vars=[f'L-{i}' for i in range(1, 6)] + ['L-7', 'L-8'],
              var_name='trap',
              value_name='temp_count')
        .rename(columns={'datetime': 'date', 
                         'Cloud': 'cloud',
                         'Temp': 'tmp',
                         'Wind': 'wind_dir',
                         'temp_count': 'count'})
        .sort_values(by=['date', 'trap'])
        .reset_index(drop=True)
    )
    # add year, month & day columns for easier grouping
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    # add trap direction and actual name of species
    df['trap_dir'] = df['trap'].apply(get_trap_direction)
    df['species'] = s
    # re-order columns in intuitive way
    column_order = [10, 0, 6, 7, 8, 4, 9, 5, 2, 1, 3]
    df = df[df.columns[column_order]]
    # merge all species into single dataframe
    all_species = pd.concat([all_species, df])

count_by_trap = all_species.reset_index(drop=True)
count_by_trap.to_csv('../../data/all_species_count_by_trap.csv', index=False)

print('some stats:')
display(count_by_trap.describe())
print('preview of data:')
display(count_by_trap.head())

some stats:


,day,month,year,count,tmp
count,880425.000000,880425.000000,880425.000000,196074.000000,652995.000000
mean,15.720930,7.027907,2001.000000,0.237018,16.141823
std,8.864162,2.011402,11.254635,6.380251,6.425214
min,1.000000,4.000000,1982.000000,0.000000,-1.000000
25%,8.000000,5.000000,1991.000000,0.000000,12.000000
50%,16.000000,7.000000,2001.000000,0.000000,16.000000
75%,23.000000,9.000000,2011.000000,0.000000,20.000000
max,31.000000,11.000000,2020.000000,1328.000000,232.000000


preview of data:


,species,date,day,month,year,trap,trap_dir,count,tmp,cloud,wind_dir
0,Vanessa atalanta,1982-04-01,1,4,1982,L-1,south,NaN,NaN,0,W
1,Vanessa atalanta,1982-04-01,1,4,1982,L-2,north,NaN,NaN,0,W
2,Vanessa atalanta,1982-04-01,1,4,1982,L-3,north,NaN,NaN,0,W
3,Vanessa atalanta,1982-04-01,1,4,1982,L-4,south,NaN,NaN,0,W
4,Vanessa atalanta,1982-04-01,1,4,1982,L-5,north,NaN,NaN,0,W


In [4]:
# create another dataframe with daily total counts (all traps combined)
count_daily = count_by_trap.drop(columns = ['trap', 'trap_dir'])
count_daily['count'] = count_daily.groupby('date')['count'].transform('sum', min_count=1)
count_daily = count_daily.drop_duplicates().reset_index(drop=True)
count_daily.to_csv('../../data/all_species_count_daily.csv', index=False)

count_daily

,species,date,day,month,year,count,tmp,cloud,wind_dir
0,Vanessa atalanta,1982-04-01,1,4,1982,NaN,NaN,0,W
1,Vanessa atalanta,1982-04-02,2,4,1982,NaN,NaN,0,W
2,Vanessa atalanta,1982-04-03,3,4,1982,NaN,NaN,1/2,NE-E
3,Vanessa atalanta,1982-04-04,4,4,1982,NaN,NaN,1/2,W-NW
4,Vanessa atalanta,1982-04-05,5,4,1982,NaN,NaN,0,NW-W
...,...,...,...,...,...,...,...,...,...
125770,Pararge aegeria,2020-10-28,28,10,2020,NaN,NaN,NaN,NaN
125771,Pararge aegeria,2020-10-29,29,10,2020,NaN,NaN,NaN,NaN
125772,Pararge aegeria,2020-10-30,30,10,2020,NaN,NaN,NaN,NaN
125773,Pararge aegeria,2020-10-31,31,10,2020,NaN,NaN,NaN,NaN


### UNIQUE VALUES

Let's see which (categorical & continuous) variables they've used to classify wind direction, cloud cover & temperature:

In [30]:
for column in ['tmp', 'cloud', 'wind_dir']:
    print(f'unique {column} values & counts:')
    print(count_daily[column].value_counts(),'\n')

unique tmp values & counts:
 17.0     6885
 20.0     6690
 18.0     6570
 15.0     6225
 16.0     6015
 13.0     5685
 12.0     5010
 14.0     4710
 21.0     4155
 19.0     4005
 22.0     3960
 10.0     3840
 23.0     3780
 11.0     3315
 7.0      2985
 8.0      2820
 9.0      2565
 24.0     2325
 25.0     2085
 6.0      1935
 26.0     1515
 5.0      1260
 27.0     1185
 28.0      825
 4.0       720
 29.0      465
 3.0       435
 2.0       360
 30.0      345
 31.0      285
 32.0      120
 1.0        60
 33.0       45
 34.0       30
 0.0        30
-1.0        15
 232.0      15
 35.0       15
Name: tmp, dtype: int64 

unique cloud values & counts:
1/2    31335
3/4    20430
0      19905
1      14891
1/4    11295
1         35
С         15
25        15
1`        15
ф         14
Name: cloud, dtype: int64 

unique wind_dir values & counts:
W          15435
NW          7035
N           5835
E           5640
N-NW        4995
           ...  
NW-N-SW       15
SW-NE-N       15
N-SW-S        15
W-

In [35]:
# additionally, print wind values to file 
wind_dir = count_daily['wind_dir'].value_counts()
df_wind_dir = pd.DataFrame(zip(wind_dir.index, wind_dir))
df_wind_dir.columns = ['direction', 'count']
display(df_wind_dir)
df_wind_dir.to_csv('../unique_wind_directions.txt', sep='\t', index=False)

,direction,count
0,W,15435
1,NW,7035
2,N,5835
3,E,5640
4,N-NW,4995
...,...,...
217,NW-N-SW,15
218,SW-NE-N,15
219,N-SW-S,15
220,W-S-NW,15
